In [1]:
import os
import cv2
from lxml import etree
import xml.etree.cElementTree as ET
import numpy as np

In [4]:
def create_object_dict (name,x_tl,y_tl,x_br,y_br,cls):
    """
    #argument:
        name: name img
        x_tl: x top left
        y_tl: y top left
        x_br: x bottom right
        y_br: y bottom right
    #return:
        obj = {'name':name,'annotation':[{'x_tl':10,'y_tl':10,'x_br':100,'y_br':100,'cls':cls}]};
    """
    obj = {'name':name,'annotation':[{'x_tl':x_tl,'y_tl':y_tl,'x_br':x_br,'y_br':y_br,'cls':cls}]}
    return obj

def create_annotation(x_tl,y_tl,x_br,y_br,cls):
    return {'x_tl':x_tl,'y_tl':y_tl,'x_br':x_br,'y_br':y_br,'cls':cls}

def convert_class(cls):
    """
    0:prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16] (circular, white ground with red border)
    1:mandatory = [33, 34, 35, 36, 37, 38, 39, 40] (circular, blue ground)
    2:danger = [11, 18, 19, 20 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31] (triangular, white ground with red border)
    3:other = [12, 13, 14, 17, 32, 41, 42]
    """
    cls = int(cls)
    if cls in [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]:
        return 'prohibitory'
    if cls in [33, 34, 35, 36, 37, 38, 39, 40]:
        return 'mandatory'
    if cls in [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]:
        return 'danger'
    if cls in [12, 13, 14, 17, 32, 41, 42]:
        return 'other'
    else:
        return 'undefine'
    
def convert_annotation_from_txt(path_file):
    """
    #return:
    form data:
        data = [{'name':'img','anotation':[ {'x_tf':10,'y_tf':10,'x_br':100,'y_br':100},
                                                    {'x_tf':10,'y_tf':10,'x_br':100,'y_br':100}  ]},
        {'name':'img1',,'anotation':{'x_tf':10,'y_tf':10,'x_br':100,'y_br':100}} ]

    """
    with open(path_file) as f:
        data = f.readlines();
        f.close();
    im_names =[]
    objects_inside_img = []
    clses = []
    for i,row in enumerate(data):
        row = row[:-1] #remove "\n" at the end row; ex: "00001.ppm;386;494;442;552;38\n"
        info_img = row.split(";")
        im_name = info_img[0]
        x_tl = info_img[1]
        y_tl = info_img[2]
        x_br = info_img[3]
        y_br = info_img[4]
        cls = info_img[5]
        if im_name in im_names:
            index = im_names.index(im_name)
            annotation = create_annotation(x_tl,y_tl,x_br,y_br,cls)
            objects_inside_img[index]['annotation'].append(annotation)
        else:
            im_names.append(im_name)
            obj = create_object_dict(im_name,x_tl,y_tl,x_br,y_br,cls)
            objects_inside_img.append(obj)
            
    return objects_inside_img


path =r"..\dataVN\Train\train_annotation.txt"
object_dict = convert_annotation_from_txt(path)
print(object_dict[1])


{'name': '00001.ppm', 'annotation': [{'x_tl': '949', 'y_tl': '214', 'x_br': '990', 'y_br': '254', 'cls': '13'}]}


In [6]:
"""
<annotation>
	<folder>VOC2007</folder>
	<filename>000001.jpg</filename>
	<size>
		<width>353</width>
		<height>500</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>dog</name>
		<pose>Left</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>48</xmin>
			<ymin>240</ymin>
			<xmax>195</xmax>
			<ymax>371</ymax>
		</bndbox>
	</object>
	<object>
		<name>person</name>
		<pose>Left</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>8</xmin>
			<ymin>12</ymin>
			<xmax>352</xmax>
			<ymax>498</ymax>
		</bndbox>
	</object>
</annotation>

"""
def write_xml(folder,object_dict, savedir):
    """
     object_dict = {'name':'name','annotation':[{'x_tl':10,'y_tl':10,'x_br':100,'y_br':100,'cls':cls}]};
    """
    if not os.path.isdir(savedir):
        os.mkdir(savedir)
    im_path = os.path.join(folder,object_dict['name'])
    image = cv2.imread(im_path)
    print(image.shape)
    height, width, depth = image.shape

    annotation = ET.Element('annotation')
    ET.SubElement(annotation, 'folder').text = folder
    ET.SubElement(annotation, 'filename').text = object_dict['name']
    ET.SubElement(annotation, 'segmented').text = '0'
    size = ET.SubElement(annotation, 'size')
    ET.SubElement(size, 'width').text = str(width)
    ET.SubElement(size, 'height').text = str(height)
    ET.SubElement(size, 'depth').text = str(depth)
    
    for obj in object_dict['annotation']:
        ob = ET.SubElement(annotation, 'object')
#         cls_name = convert_class(obj['cls'])
        cls_name = obj['cls']
        ET.SubElement(ob, 'name').text = cls_name
        ET.SubElement(ob, 'pose').text = 'Unspecified'
        ET.SubElement(ob, 'truncated').text = '0'
        ET.SubElement(ob, 'difficult').text = '0'
        bbox = ET.SubElement(ob, 'bndbox')
        ET.SubElement(bbox, 'xmin').text = obj['x_tl']
        ET.SubElement(bbox, 'ymin').text = obj['y_tl']
        ET.SubElement(bbox, 'xmax').text = obj['x_br']
        ET.SubElement(bbox, 'ymax').text = obj['y_br']
    
    xml_str = ET.tostring(annotation)
    root = etree.fromstring(xml_str)
    xml_str = etree.tostring(root, pretty_print=True)
    save_path = os.path.join(savedir, object_dict['name'].replace('ppm', 'xml'))
    with open(save_path, 'wb') as temp_xml:
        temp_xml.write(xml_str)


if __name__ == '__main__':
    """
    for testing
    """

    folder = r'..\dataVN\Train'
    savedir = os.path.join(os.path.dirname(folder),'annotation')

    path =r"..\dataVN\Train\train_annotation.txt"
    objects_dict  =convert_annotation_from_txt(path)
    for i,object_dict in enumerate(objects_dict):
#         print(i)
        write_xml(folder,object_dict,savedir)
    

(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280, 3)
(720, 1280

In [16]:
a = {'x':[{'1':1,'2':2},{'1':1,'2':2},{'1':1,'2':2}]}
for i in a['x']:
    print(i)
#     for b,c in i:
#         print('b:',b,";c: ",c)

{'1': 1, '2': 2}
{'1': 1, '2': 2}
{'1': 1, '2': 2}


In [30]:
os.path.dirname(r'..\detect_data\train')

'..\\detect_data'

In [47]:
p= r'C:\Users\buico\traffic_sign_project\detect_data\train\TrainIJCNN2013\13\00012.ppm'
im = cv2.imread(p)
cv2.imshow('im',im)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
img = cv2.imread(r"C:\Users\Bui Huy\Downloads\00423.ppm")
tl = (651,58)
br = (755,171)
frame = cv2.rectangle(img, tl, br, (255,0,0), 2)
cv2.imshow('xx',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
def read_labels(path):
    with open(path) as f:
        data = f.readlines();
        f.close();
    clses = []
    for i,row in enumerate(data):
        row = row[:-1] #remove "\n" at the end row; ex: "00001.ppm;386;494;442;552;38\n"
        info_img = row.split(";")
        im_name = info_img[0]
        cls = info_img[5]
        clses.append(cls)
    return clses
label = read_labels(r'..\dataVN\Train\train_annotation.txt')

In [19]:
label = read_labels(r'..\dataVN\Train\train_annotation.txt')
label = np.asarray(label)
label = label.astype(np.int)
print(label.shape)
# num_label = np.bincount(label)
# label = label[num_label]
label = np.unique(label)
label

(672,)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [20]:
5*(28+5)

165